# Movies
Max Davish, Christian Miljkovic, & Rosanna Rodriguez

Projects in Programming and Data Science

See project proposal [here](https://www.maxdavish.com/s/Moneyballing-the-Movies-Project-Proposal.htm).

## Part I: Scraping Box Office Data
The first step is to get the most important data - the box office data for movies. A key problem, however, is simply _finding_ all the movies on BoxOfficeMojo's website. To solve this problem, we'll visit the [yearly box office](http://www.boxofficemojo.com/yearly/) page, which we can then use to visit another set of [pages that list the top movies](http://www.boxofficemojo.com/yearly/chart/?yr=2017&p=.htm) from each year. 

From this list let's choose to get data on, say, the top 100 movies at the box office from every year between 1990 and 2017.

Now let's define some useful functions and variables that we need to scrape the data.

In [1]:
#A function for getting HTML pages:
def GetHTML(URL):
    import requests
    from lxml import html 
    return html.fromstring((requests.get(URL,stream=True)).text,)

#Here are some useful variables to define for future functions:
#These help us locate particular data points in the data tables within the HTML pages above.
xp_partA = '//td//td//tr[(((count(preceding-sibling::*) + 1) = '
xp_partB = ') and parent::*)]//td[(((count(preceding-sibling::*) + 1) = '
xp_partC = ') and parent::*)]'

title_index  = 2
studio_index = 3
boxoffice_index = 4
theaters_index = 5
openingweekend_index = 6
date_index = 8

#This is a separate function needed to get the URLs embedded in some of those data points.
#NOTE: It requires the webpage to be defined OUTSIDE the function, which may seem odd, but...

    #It's important to understand the reason behind our defining the URL outside of the function - 
    #By doing so, getPoint function XPath queries a static object in our data ... 
    # ... rather than re-requerying the URL for every single point
    #This drastically reduces the amount of time this process will take for the entire website
    #(Although it will still be highly time-consuming)

def getURL(row):
    link_xp = '//td//td//tr[(((count(preceding-sibling::*) + 1) = '+str(row+1)+') and parent::*)]//b//a/@href'
    link = webpage.xpath(link_xp)[0]
    root = 'http://www.boxofficemojo.com'
    return root+link

#Webpage is still defined outside the function:
def getPoint(row,column):
    xp = xp_partA + str(row) + xp_partB + str(column) + xp_partC
    return webpage.xpath(xp)[0].text_content()

#Here, both the webpage AND the year are defined outside of the function. 
#This is also appropriate for the upcoming loop.
def getTable():
    table = []
    for i in range(2,102):
        newrow = {"Title": getPoint(i,title_index),
                  "URL": getURL((i-1)),
                  "Studio": getPoint(i,studio_index),
                  "Box_Office": getPoint(i,boxoffice_index),
                  "Theaters": getPoint(i,theaters_index),
                  "Opening_Weekend": getPoint(i,openingweekend_index),
                  #Like the URL, the date must be defined OUTSIDE of the function
                  "Release_Date": str(getPoint(i,date_index))+'/'+str(year),
        }
        table.append(newrow)
    return table    

Using these functions, we can easily loop through BoxOfficeMojo URLs and compile our data.
This will take a LONG time, so be patient...

_(It's roughly a 4 minute process if you choose only the top 100 movies from 1980-2017)_

In [2]:
data = []
for year in range(2017,1980, -1):
    page = 1
    url = "http://www.boxofficemojo.com/yearly/chart/?page="+str(page)+"&view=releasedate&view2=domestic&yr="+str(year)+"&p=.htm"
    webpage = GetHTML(url)
    data += getTable()
    
len(data)

3700

In [3]:
import pandas
df = pandas.DataFrame(data)
df

,Box_Office,Opening_Weekend,Release_Date,Studio,Theaters,Title,URL
0,"$481,009,028","$174,750,616",3/17/2017,BV,"4,210",Beauty and the Beast (2017),http://www.boxofficemojo.com/movies/?id=beauty...
1,"$224,572,433","$88,411,916",3/3/2017,Fox,"4,071",Logan,http://www.boxofficemojo.com/movies/?id=wolver...
2,"$194,608,240","$98,786,705",4/14/2017,Uni.,"4,329",The Fate of the Furious,http://www.boxofficemojo.com/movies/?id=furiou...
3,"$174,705,216","$53,003,468",2/10/2017,WB,"4,088",The LEGO Batman Movie,http://www.boxofficemojo.com/movies/?id=lego2.htm
4,"$172,687,155","$33,377,060",2/24/2017,Uni.,"3,143",Get Out,http://www.boxofficemojo.com/movies/?id=blumho...
5,"$165,568,393","$61,025,472",3/10/2017,WB,"3,846",Kong: Skull Island,http://www.boxofficemojo.com/movies/?id=legend...
6,"$149,206,615","$50,198,902",3/31/2017,Fox,"3,829",The Boss Baby,http://www.boxofficemojo.com/movies/?id=bossba...
7,"$138,124,620","$40,010,975",1/20/2017,Uni.,"3,373",Split,http://www.boxofficemojo.com/movies/?id=split2...
8,"$114,434,010","$46,607,250",2/10/2017,Uni.,"3,714",Fifty Shades Darker,http://www.boxofficemojo.com/movies/?id=fiftys...
9,"$92,029,184","$30,436,123",2/10/2017,LG/S,"3,113",John Wick: Chapter Two,http://www.boxofficemojo.com/movies/?id=johnwi...


The next step is to _clean_ that data (which mostly involves changing data types). 

Let's write a function that turns messy strings into floats.

**Reminder:** For error handling, always coerce to None instead of "N/A", so that the database treats them as NULL values instead of varchars.

In [5]:
def CleanNumber(n):
    if n == "N/A" or n == None:
        return None
    else:
        return float((n.strip("$")).replace(",",""))

df["Box_Office"] = df["Box_Office"].apply(CleanNumber)
df["Opening_Weekend"] = df["Opening_Weekend"].apply(CleanNumber)
df["Theaters"] = df["Theaters"].apply(CleanNumber)
df["Release_Date"] = pandas.to_datetime(df["Release_Date"],errors='coerce')
df

,Box_Office,Opening_Weekend,Release_Date,Studio,Theaters,Title,URL
0,481009028.0,174750616.0,2017-03-17,BV,4210.0,Beauty and the Beast (2017),http://www.boxofficemojo.com/movies/?id=beauty...
1,224572433.0,88411916.0,2017-03-03,Fox,4071.0,Logan,http://www.boxofficemojo.com/movies/?id=wolver...
2,194608240.0,98786705.0,2017-04-14,Uni.,4329.0,The Fate of the Furious,http://www.boxofficemojo.com/movies/?id=furiou...
3,174705216.0,53003468.0,2017-02-10,WB,4088.0,The LEGO Batman Movie,http://www.boxofficemojo.com/movies/?id=lego2.htm
4,172687155.0,33377060.0,2017-02-24,Uni.,3143.0,Get Out,http://www.boxofficemojo.com/movies/?id=blumho...
5,165568393.0,61025472.0,2017-03-10,WB,3846.0,Kong: Skull Island,http://www.boxofficemojo.com/movies/?id=legend...
6,149206615.0,50198902.0,2017-03-31,Fox,3829.0,The Boss Baby,http://www.boxofficemojo.com/movies/?id=bossba...
7,138124620.0,40010975.0,2017-01-20,Uni.,3373.0,Split,http://www.boxofficemojo.com/movies/?id=split2...
8,114434010.0,46607250.0,2017-02-10,Uni.,3714.0,Fifty Shades Darker,http://www.boxofficemojo.com/movies/?id=fiftys...
9,92029184.0,30436123.0,2017-02-10,LG/S,3113.0,John Wick: Chapter Two,http://www.boxofficemojo.com/movies/?id=johnwi...


We also need to add a few extra fields to this table:
- **BoxOfficeMojo ID**, which we can extract from the URL with a RegEx
- **IMDB_ID**, which we can use the OMDB API to find
- **Production Budget**, which we'll also find on BoxOfficeMojo, using the BoxOffice ID 

Let's write functions to acquire those variables.

In [16]:
def Extract_BoxOfficeID(URL):
    import re
    regex = re.compile('(http:\/\/www.boxofficemojo.com\/movies\/\?id=)((\w)+)(.htm)')
    matches = regex.finditer(URL)
    for match in matches:
        return match.group(2)
    
def GetProdBudget(ID):
    try:
        url = 'http://www.boxofficemojo.com/movies/?page=daily&view=chart&id='+ID+'.htm'
    except TypeError:
        return None
    page = GetHTML(url)
    budget_xp = '//center//tr[(((count(preceding-sibling::*) + 1) = 4) and parent::*)]//b'
    budget = page.xpath(budget_xp)
    try:
        if budget[1].text_content() == 'N/A':
            return None
        else:
            return budget[1].text_content()
    except IndexError:
        return None
    
def GetIMDB_ID(title):
        #Might want to add a 'year' parameter later on, but we'll assess accuracy.
    import requests
    omdb_url = 'http://www.omdbapi.com/?'
    parameters = {'s': title, 'type':'movie'} 
    try:
        return requests.get(url = omdb_url, params = parameters).json()["Search"][0]["imdbID"]
    except KeyError:
        return None
    except ValueError: 
        #Should also catch pesky JSONDecoderError
        return None

Now we'll add all three columns to the dataframe. Make sure to run these cells separately - otherwise you'll run into server overload errors. _(You might anyway...)_

In [7]:
df["BoxOfficeID"] = df["URL"].apply(Extract_BoxOfficeID)

In [17]:
df["Production_Budget"] = df["BoxOfficeID"].apply(GetProdBudget)

**Warning:** This cell takes a _very_ long time to run (15 minutes+), so try not to run it more than once, if you can avoid it.

In [74]:
df["IMDB_ID"] = df["Title"].apply(GetIMDB_ID)
df

,Box_Office,Opening_Weekend,Release_Date,Studio,Theaters,Title,URL,BoxOfficeID,Production_Budget,IMDB_ID
0,481009028.0,174750616.0,2017-03-17,BV,4210.0,Beauty and the Beast (2017),http://www.boxofficemojo.com/movies/?id=beauty...,beautyandthebeast2017,160000000.0,None
1,224572433.0,88411916.0,2017-03-03,Fox,4071.0,Logan,http://www.boxofficemojo.com/movies/?id=wolver...,wolverine2017,97000000.0,tt3315342
2,194608240.0,98786705.0,2017-04-14,Uni.,4329.0,The Fate of the Furious,http://www.boxofficemojo.com/movies/?id=furiou...,furious8,250000000.0,tt4630562
3,174705216.0,53003468.0,2017-02-10,WB,4088.0,The LEGO Batman Movie,http://www.boxofficemojo.com/movies/?id=lego2.htm,lego2,80000000.0,tt4116284
4,172687155.0,33377060.0,2017-02-24,Uni.,3143.0,Get Out,http://www.boxofficemojo.com/movies/?id=blumho...,blumhouse2,4500000.0,tt5052448
5,165568393.0,61025472.0,2017-03-10,WB,3846.0,Kong: Skull Island,http://www.boxofficemojo.com/movies/?id=legend...,legendary2016,185000000.0,tt3731562
6,149206615.0,50198902.0,2017-03-31,Fox,3829.0,The Boss Baby,http://www.boxofficemojo.com/movies/?id=bossba...,bossbaby,NaN,tt3874544
7,138124620.0,40010975.0,2017-01-20,Uni.,3373.0,Split,http://www.boxofficemojo.com/movies/?id=split2...,split2017,9000000.0,tt4972582
8,114434010.0,46607250.0,2017-02-10,Uni.,3714.0,Fifty Shades Darker,http://www.boxofficemojo.com/movies/?id=fiftys...,fiftyshadesdarker,55000000.0,tt4465564
9,92029184.0,30436123.0,2017-02-10,LG/S,3113.0,John Wick: Chapter Two,http://www.boxofficemojo.com/movies/?id=johnwi...,johnwick2,NaN,None


Unfortunately, this process didn't work perfectly, and some of data is simply unavailable - many movies are missing a BoxOffice or IMDB ID or lack a production budget. Later on, we can revisit or functions to try to minimize these sources of error. 

Before we get to databasing, let's do one last cleaning measure - **converting production budgets from strings to floats.** We'll write a regex function to do this as well.




In [58]:
#Write the function
def CleanProdBudget(n):
    if n == None:
        return None
    else:
        import re
        regex = re.compile('([\d]+[\.]?[\d]?)( million)')
        matches = regex.finditer(n)
        for match in matches:
            return (float(match.group(1))*1000000)

In [60]:
df["Production_Budget"] = df["Production_Budget"].apply(CleanProdBudget)
df

,Box_Office,Opening_Weekend,Release_Date,Studio,Theaters,Title,URL,BoxOfficeID,Production_Budget
0,481009028.0,174750616.0,2017-03-17,BV,4210.0,Beauty and the Beast (2017),http://www.boxofficemojo.com/movies/?id=beauty...,beautyandthebeast2017,160000000.0
1,224572433.0,88411916.0,2017-03-03,Fox,4071.0,Logan,http://www.boxofficemojo.com/movies/?id=wolver...,wolverine2017,97000000.0
2,194608240.0,98786705.0,2017-04-14,Uni.,4329.0,The Fate of the Furious,http://www.boxofficemojo.com/movies/?id=furiou...,furious8,250000000.0
3,174705216.0,53003468.0,2017-02-10,WB,4088.0,The LEGO Batman Movie,http://www.boxofficemojo.com/movies/?id=lego2.htm,lego2,80000000.0
4,172687155.0,33377060.0,2017-02-24,Uni.,3143.0,Get Out,http://www.boxofficemojo.com/movies/?id=blumho...,blumhouse2,4500000.0
5,165568393.0,61025472.0,2017-03-10,WB,3846.0,Kong: Skull Island,http://www.boxofficemojo.com/movies/?id=legend...,legendary2016,185000000.0
6,149206615.0,50198902.0,2017-03-31,Fox,3829.0,The Boss Baby,http://www.boxofficemojo.com/movies/?id=bossba...,bossbaby,NaN
7,138124620.0,40010975.0,2017-01-20,Uni.,3373.0,Split,http://www.boxofficemojo.com/movies/?id=split2...,split2017,9000000.0
8,114434010.0,46607250.0,2017-02-10,Uni.,3714.0,Fifty Shades Darker,http://www.boxofficemojo.com/movies/?id=fiftys...,fiftyshadesdarker,55000000.0
9,92029184.0,30436123.0,2017-02-10,LG/S,3113.0,John Wick: Chapter Two,http://www.boxofficemojo.com/movies/?id=johnwi...,johnwick2,NaN


We're finally ready to insert this dataframe into a database. This dataframe will form the backbone of the database - the central table to which all others are related.

In [67]:
#Connection to database:
import MySQLdb as mdb
import sys

con = mdb.connect(host = 'localhost', 
                  user = 'root', 
                  passwd = 'dwdstudent2015', 
                  charset='utf8', use_unicode=True);

In [68]:
#This code deletes the database if it exists.
    #This is a useful cell to have while designing database, but now that the code is finished I will comment this part out.
db_name = 'Movies'
drop_db_query = 'DROP DATABASE IF EXISTS {db}'.format(db=db_name)
cursor = con.cursor()
cursor.execute(drop_db_query)
cursor.close()

#Create the database:
create_db_query = "CREATE DATABASE IF NOT EXISTS {db} DEFAULT CHARACTER SET 'utf8'".format(db=db_name)
cursor = con.cursor()
cursor.execute(create_db_query)
cursor.close()

In [76]:
cursor = con.cursor()
table_name = 'Movies'
create_table_query = '''CREATE TABLE IF NOT EXISTS {db}.{table}
                        (Box_Office varchar(250),
                        Opening_Weekend float,
                        Release_Date datetime,
                        Studio varchar(250),
                        Theaters int,
                        Title varchar(250),
                        BoxOfficeID varchar(250),
                        Production_Budget float,
                        IMDB_ID varchar(250),
                        PRIMARY KEY(BoxOfficeID)
                        )'''.format(db=db_name, table=table_name)
cursor.execute(create_table_query)
cursor.close()

In [77]:
cursor = con.cursor()
table_name = 'Movies'
db_name = 'Movies'
insert_query_template = '''INSERT IGNORE INTO {db}.{table}(Box_Office,
                        Opening_Weekend,
                        Release_Date,
                        Studio,
                        Theaters,
                        Title,
                        BoxOfficeID,
                        Production_Budget,
                        IMDB_ID)
                        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)'''.format(db=db_name, table=table_name)
for i in range(0,len(df)):
    try:
        query_parameters = (df["Box_Office"][i],
                        df["Opening_Weekend"][i],
                        df["Release_Date"][i],
                        df["Studio"][i],
                        df["Theaters"][i],
                        df["Title"][i],
                        df["BoxOfficeID"][i],
                        df["Production_Budget"][i],
                        df["IMDB_ID"][i])
        cursor.execute(insert_query_template,query_parameters)
    except KeyError:
        continue

con.commit()
cursor.close()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:25: Warning: (1265, "Data truncated for column 'Production_Budget' at row 1")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:25: Warning: (1265, "Data truncated for column 'Opening_Weekend' at row 1")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:25: Warning: (1048, "Column 'BoxOfficeID' cannot be null")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:25: Warning: (1062, "Duplicate entry '' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:25: Warning: (1366, "Incorrect integer value: 'nan' for column 'Theaters' at row 1")


There you have it! We have the first table of our database. Up next - the foreign and day-by-day tables.